In [ ]:
import pandas as pd
import numpy as  np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam


In [ ]:
data = pd.read_csv('dataset/Resep_Dengan_Kolom_Alergen.csv')
data.head()

,name,food_name,Images,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,CarbohydrateContent,ProteinContent,RecipeServings,RecipeInstructions,Unnamed: 13,is_seafood,is_nut,is_lactose
0,1,Cabbage Soup,https://img.sndimg.com/food/image/upload/w_555...,PT30M,PT20M,PT50M,"""plain tomato juice"", ""cabbage"", ""onion"", ""car...",103.6,0.4,25.1,4.3,4,"Mix everything together and bring to a boil."",...",NaN,0,0,0
1,2,Buttermilk Pie With Gingersnap Crumb Crust,https://img.sndimg.com/food/image/upload/w_555...,PT50M,PT30M,PT1H20M,"""sugar"", ""margarine"", ""egg"", ""flour"", ""salt"", ...",228.0,7.1,37.5,4.2,8,"""Preheat oven to 350°F."", ""Make pie crust, usi...",NaN,0,0,1
2,3,Lou's Fabulous Bruschetta,https://img.sndimg.com/food/image/upload/w_555...,PT10M,PT30M,PT40M,"""French baguette"", ""butter"", ""garlic powder"", ...",221.5,6.5,32.5,8.1,8,"Cut baguette into slices."", ""Butter and then s...",NaN,0,0,1
3,4,Brownie Cheesecake Torte,https://img.sndimg.com/food/image/upload/w_555...,PT55M,PT35M,PT1H30M,"""instant coffee granules"", ""cinnamon"", ""sugar""...",122.1,4.9,14.4,5.7,12,"""Preheat oven to 425 degrees F."", ""Combine fir...",NaN,0,0,1
4,5,Brazilian Empadinhas,https://img.sndimg.com/food/image/upload/w_555...,PT2H37M,PT1H,PT3H37M,"""butter"", ""flour"", ""salt"", ""margarine"", ""milk""...",329.7,16.7,35.6,9.3,8,"""DOUGH Cut butter into salt and flour."", ""Stir...",NaN,1,0,1


In [ ]:
dataset=data.copy()
columns=['name', 'food_name', 'Images',
         'CookTime', 'PrepTime', 'TotalTime',
         'RecipeIngredientParts', 'Calories', 'FatContent',
         'CarbohydrateContent', 'ProteinContent', 'RecipeServings',
         'RecipeInstructions','is_seafood','is_nut', 'is_lactose'
         ]
dataset=dataset[columns]
dataset.head()

,name,food_name,Images,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,CarbohydrateContent,ProteinContent,RecipeServings,RecipeInstructions,is_seafood,is_nut,is_lactose
0,1,Cabbage Soup,https://img.sndimg.com/food/image/upload/w_555...,PT30M,PT20M,PT50M,"""plain tomato juice"", ""cabbage"", ""onion"", ""car...",103.6,0.4,25.1,4.3,4,"Mix everything together and bring to a boil."",...",0,0,0
1,2,Buttermilk Pie With Gingersnap Crumb Crust,https://img.sndimg.com/food/image/upload/w_555...,PT50M,PT30M,PT1H20M,"""sugar"", ""margarine"", ""egg"", ""flour"", ""salt"", ...",228.0,7.1,37.5,4.2,8,"""Preheat oven to 350°F."", ""Make pie crust, usi...",0,0,1
2,3,Lou's Fabulous Bruschetta,https://img.sndimg.com/food/image/upload/w_555...,PT10M,PT30M,PT40M,"""French baguette"", ""butter"", ""garlic powder"", ...",221.5,6.5,32.5,8.1,8,"Cut baguette into slices."", ""Butter and then s...",0,0,1
3,4,Brownie Cheesecake Torte,https://img.sndimg.com/food/image/upload/w_555...,PT55M,PT35M,PT1H30M,"""instant coffee granules"", ""cinnamon"", ""sugar""...",122.1,4.9,14.4,5.7,12,"""Preheat oven to 425 degrees F."", ""Combine fir...",0,0,1
4,5,Brazilian Empadinhas,https://img.sndimg.com/food/image/upload/w_555...,PT2H37M,PT1H,PT3H37M,"""butter"", ""flour"", ""salt"", ""margarine"", ""milk""...",329.7,16.7,35.6,9.3,8,"""DOUGH Cut butter into salt and flour."", ""Stir...",1,0,1


In [ ]:
extracted_data=dataset.copy()
extracted_data = extracted_data[extracted_data.Images != 'character(0)']
extracted_data.to_csv('Resep_Dengan_Kolom_Alergen.csv', index=False)

In [31]:
from sklearn.metrics.pairwise import euclidean_distances

# Define the recommendation function
def get_recommendations(input_calories, input_seafood, input_nut, input_lactose, top_k):
    # Calculate the Euclidean distances between input calories and all recipes
    distances = euclidean_distances(extracted_data[['Calories']], [[input_calories]])

    # Add distances as a new column in the dataset
    extracted_data['Distance'] = distances

    if input_seafood == 1:
      filtered_recipes = extracted_data[
        (extracted_data['is_seafood'] == 0)
      ]

    if input_nut == 1:
      filtered_recipes = extracted_data[
        (extracted_data['is_nut'] == 0)
      ]

    if input_lactose == 1:
      filtered_recipes = extracted_data[
        (extracted_data['is_lactose'] == 0)
      ]
    # Sort recipes based on the distances in ascending order
    sorted_recipes = filtered_recipes.sort_values('Distance')

    # Select top k recipes
    top_recipes = sorted_recipes.head(top_k)

    return top_recipes

# Test the recommendation system
input_calories = 2000  # Example input calories
input_lactose = 0     # Example input for lactose (0 for no lactose, 1 for lactose)
input_seafood = 0     # Example input for seafood (0 for no seafood, 1 for seafood)
input_nut = 1         # Example input for nut (0 for no nut, 1 for nut)
top_k = 100
recommended_recipes = get_recommendations(input_calories, input_seafood, input_nut, input_lactose, top_k=10)

recommended_recipes.head(top_k)

,name,food_name,Images,CookTime,PrepTime,TotalTime,RecipeIngredientParts,Calories,FatContent,CarbohydrateContent,ProteinContent,RecipeServings,RecipeInstructions,is_seafood,is_nut,is_lactose,Distance
305,306,Sushi-Style Roll-Ups,https://img.sndimg.com/food/image/upload/w_555...,PT4H30M,PT5M,PT4H35M,"""medium-grain white rice"", ""seasoned rice vine...",1281.6,26.1,229.6,28.1,2,"""In medium bowl, combine cooked rice and vineg...",0,0,1,718.4
675,676,Campbell's Orange Beef,https://img.sndimg.com/food/image/upload/w_555...,PT20M,PT10M,PT30M,"""boneless beef top sirloin steak"", ""green pepp...",1098.1,26.9,171.7,37.4,4,"SLICE beef into very thin strips."", ""In medium...",0,0,0,901.9
324,325,Biff's Sweet Sour Shrimp and Pasta,https://img.sndimg.com/food/image/upload/w_555...,PT8M,PT5M,PT13M,"""baby shrimp"", ""dry pasta"", ""tomatoes"", ""catsu...",921.4,30.6,117.8,43.3,1,"""Cook pasta in two quarts of salted water unti...",1,0,1,1078.6
261,262,Indian Tacos,https://img.sndimg.com/food/image/upload/w_555...,PT15M,PT15M,PT30M,"""lean ground beef"", ""onion"", ""salt"", ""pepper"",...",871.5,25.4,108.6,49.8,5,"""Combine meat, onion, salt, pepper and Tabasco...",0,0,1,1128.5
486,487,Plum Dumplings - Blommeboller,https://img.sndimg.com/food/image/upload/w_555...,PT20M,PT5M,PT25M,"""potatoes"", ""flour"", ""sugar"", ""eggs"", ""salt"", ...",852.7,17.5,154.7,20.4,6,"""Cook and peel the potatoes."", ""Mash in a larg...",0,0,1,1147.3
956,957,Lemon Chicken and Asparagus Over Rice,https://img.sndimg.com/food/image/upload/w_555...,PT15M,PT15M,PT30M,"""olive oil"", ""boneless skinless chicken breast...",839.2,21.0,91.7,62.4,2,"In a large non-stick skillet, heat oil over me...",0,0,0,1160.8
471,472,Fruited Turkey Breast,https://img.sndimg.com/food/image/upload/w_555...,PT1H45M,PT10M,PT1H55M,"""turkey breast"", ""thyme"", ""rosemary"", ""onion"",...",823.3,24.5,68.2,78.1,4,"""Preheat the oven to 450 F."", ""Place the turke...",0,0,0,1176.7
579,580,Linguine with Red Clam Sauce,https://img.sndimg.com/food/image/upload/w_555...,PT15M,PT10M,PT25M,"""butter"", ""olive oil"", ""garlic"", ""diced tomato...",817.3,25.8,99.3,45.2,4,Heat butter and olive oil in pan over medium h...,1,0,1,1182.7
300,301,Easy Chicken Cacciatore,https://img.sndimg.com/food/image/upload/w_555...,PT1H,PT25M,PT1H25M,"""chicken"", ""seasoned flour"", ""olive oil"", ""gar...",748.7,52.7,22.2,46.5,6,"Roll chicken in flour and brown in olive oil.""...",0,0,0,1251.3
111,112,Hot and Spicy Almonds,https://img.sndimg.com/food/image/upload/w_555...,PT20M,PT5M,PT25M,"""unsalted butter"", ""Worcestershire sauce"", ""cu...",734.4,66.0,24.2,23.2,4,"Preheat oven to 350 degrees."", ""Melt the butte...",0,0,1,1265.6
